In [1]:
!pip install fastapi uvicorn pyngrok joblib requests nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.4 MB/s eta 0:00:00


In [2]:
!wget -O intent_classification.joblib https://github.com/Muhammad-Ikhwan-Fathulloh/Hands-On-NLP-Super-Class-Batch2/raw/main/Text_Classifier/intent_classification.joblib

--2025-01-13 11:09:29--  https://github.com/Muhammad-Ikhwan-Fathulloh/Hands-On-NLP-Super-Class-Batch2/raw/main/Text_Classifier/intent_classification.joblib
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Muhammad-Ikhwan-Fathulloh/Hands-On-NLP-Super-Class-Batch2/main/Text_Classifier/intent_classification.joblib [following]
--2025-01-13 11:09:29--  https://raw.githubusercontent.com/Muhammad-Ikhwan-Fathulloh/Hands-On-NLP-Super-Class-Batch2/main/Text_Classifier/intent_classification.joblib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4942 (4.8K) [application/octet-stream]
Saving to: ‘intent_class

In [3]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import joblib
from pyngrok import ngrok
from threading import Thread
import uvicorn
import nest_asyncio

# Apply nest_asyncio to prevent runtime errors
nest_asyncio.apply()

# Load the trained model
model = joblib.load('intent_classification.joblib')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.5.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.5.2 when using version 1.6.0. This might lead to breaking c

In [4]:
# Initialize FastAPI
app = FastAPI()

# Define the input data structure
class TextInput(BaseModel):
    text: str

# Define the prediction endpoint
@app.post("/predict/")
async def predict_intent(input: TextInput):
    try:
        # Preprocess input text (if needed, e.g., lowercase)
        processed_text = input.text.lower()

        # Predict intent
        prediction = model.predict([processed_text])

        # Return the prediction as a JSON response
        return {"intent": prediction[0]}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")

# Define a health check endpoint
@app.get("/")
async def root():
    return {"message": "Intent Classification Model is running successfully"}

@app.get("/profile")
async def profile():
    return {"name": "Ikhwan", "description": "Learn NLP"}

In [5]:
# Tambahkan token ngrok Anda
!ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
# Run the server in a separate thread
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start ngrok tunnel
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Run the FastAPI server
thread = Thread(target=run)
thread.start()

Public URL: NgrokTunnel: "https://6a5d-35-239-141-52.ngrok-free.app" -> "http://localhost:8000"


In [7]:
import requests

# Masukkan URL ngrok yang Anda dapatkan
url = f"https://6a5d-35-239-141-52.ngrok-free.app/predict"

# Kirim request ke endpoint prediksi
data = {"text": "Hello, how are you?"}
response = requests.post(url, json=data)

# Cetak hasil prediksi
print("Response:", response.json())

INFO:     35.239.141.52:0 - "POST /predict HTTP/1.1" 307 Temporary Redirect
INFO:     35.239.141.52:0 - "POST /predict/ HTTP/1.1" 200 OK
Response: {'intent': 'greeting'}
